# 手写数字识别

## 加载相关库

In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, optimizers
import numpy as np

Init Plugin
Init Graph Optimizer
Init Kernel


## 加载相关数据

In [2]:
(train_data, test_data) = datasets.mnist.load_data()
print('train_dataset: ', train_data[0].shape, train_data[1].shape)
print('test_dataset: ', test_data[0].shape, test_data[1].shape)

train_dataset:  (60000, 28, 28) (60000,)
test_dataset:  (10000, 28, 28) (10000,)


## 建模

### DNN

In [3]:
train_x = tf.convert_to_tensor(train_data[0], dtype=tf.float32) / 255. 
test_x = tf.convert_to_tensor(test_data[0], dtype=tf.float32) / 255.

train_y = tf.convert_to_tensor(train_data[1])
test_y = tf.convert_to_tensor(test_data[1], dtype=tf.float32)

train_x = tf.reshape(train_x, (-1, 28*28))
test_x = tf.reshape(test_x, (-1, 28*28))


train_db = tf.data.Dataset.from_tensor_slices((train_x, train_y)).batch(100)
test_db = tf.data.Dataset.from_tensor_slices((test_x, test_y)).batch(100)

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2021-08-26 17:10:20.718965: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-08-26 17:10:20.719082: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [4]:
class DNN(tf.keras.Model):
    def __init__(self):
        super(DNN, self).__init__()
        self.linear_1 = layers.Dense(512, activation='relu')
        self.linear_2 = layers.Dense(256, activation='relu')
        self.linear_3 = layers.Dense(10)

    def call(self, x):
        x = self.linear_1(x)
        x = self.linear_2(x)
        x = self.linear_3(x)
        return x

model = DNN()

In [5]:
loss_function = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = optimizers.Adam()

In [6]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [7]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        prediction = model(images)
        loss = loss_function(labels, prediction)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, prediction)


@tf.function
def test_step(images, labels):
  predictions = model(images)
  t_loss = loss_function(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)
        

In [8]:
EPOCHS = 10


for epoch in range(EPOCHS):
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

    for images, labels in train_db:
        train_step(images, labels)

    for test_images, test_labels in test_db:
        test_step(test_images, test_labels)

    template = 'Epoch {}, Loss: {}, Accuracy: {}%, Test Loss: {}, Test Accuracy: {}%'
        
    print(template.format(epoch + 1,
                          train_loss.result(), 
                          train_accuracy.result() * 100,
                          test_loss.result(),
                          test_accuracy.result() * 100))
    
        

2021-08-26 17:10:21.325233: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-08-26 17:10:21.326354: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2021-08-26 17:10:21.326424: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-08-26 17:10:24.362538: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


Epoch 1, Loss: 0.23318937420845032, Accuracy: 93.05000305175781%, Test Loss: 0.11252956837415695, Test Accuracy: 96.3800048828125%
Epoch 2, Loss: 0.08812576532363892, Accuracy: 97.30333709716797%, Test Loss: 0.12395542860031128, Test Accuracy: 95.9800033569336%
Epoch 3, Loss: 0.05420693755149841, Accuracy: 98.36833190917969%, Test Loss: 0.2999819219112396, Test Accuracy: 92.72000122070312%
Epoch 4, Loss: 0.036737002432346344, Accuracy: 98.83333587646484%, Test Loss: 0.10690704733133316, Test Accuracy: 96.99000549316406%
Epoch 5, Loss: 0.028400951996445656, Accuracy: 99.0999984741211%, Test Loss: 0.11166983097791672, Test Accuracy: 97.34000396728516%
Epoch 6, Loss: 0.02575034275650978, Accuracy: 99.17666625976562%, Test Loss: 0.08645763993263245, Test Accuracy: 97.79000091552734%
Epoch 7, Loss: 0.017862997949123383, Accuracy: 99.37833404541016%, Test Loss: 0.08414957672357559, Test Accuracy: 97.9000015258789%
Epoch 8, Loss: 0.015828153118491173, Accuracy: 99.4566650390625%, Test Loss: 0

In [9]:
model.summary()

Model: "dnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                multiple                  401920    
_________________________________________________________________
dense_1 (Dense)              multiple                  131328    
_________________________________________________________________
dense_2 (Dense)              multiple                  2570      
Total params: 535,818
Trainable params: 535,818
Non-trainable params: 0
_________________________________________________________________


In [13]:
train_x = tf.convert_to_tensor(train_data[0], dtype=tf.float32) / 255. 
test_x = tf.convert_to_tensor(test_data[0], dtype=tf.float32) / 255.

train_y = tf.convert_to_tensor(train_data[1])
test_y = tf.convert_to_tensor(test_data[1], dtype=tf.float32)

train_x = tf.reshape(train_x, (-1, 28, 28, 1))
test_x = tf.reshape(test_x, (-1, 28, 28, 1))


train_db = tf.data.Dataset.from_tensor_slices((train_x, train_y)).batch(100)
test_db = tf.data.Dataset.from_tensor_slices((test_x, test_y)).batch(100)

In [10]:
class CNN(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.conv_1 = layers.Conv2D(32, (3, 3), activation='relu')
        self.conv_2 = layers.Conv2D(64, (3, 3), activation='relu')
        self.conv_3 = layers.Conv2D(64, (3, 3), activation='relu')
        self.linear_1 = layers.Dense(64, activation='relu')
        self.linear_2 = layers.Dense(10)
        self.pooling = layers.MaxPool2D((2, 2))
        self.flatten = layers.Flatten()

    def call(self, x):
        x = self.conv_1(input)
        x = self.pooling(x)
        x = self.conv_2(x)
        x = self.pooling(x)
        x = self.conv_3(x)
        x = self.flatten(x)
        x = self.linear_1(x)
        x = self.linear_2(x)
        return x

model = CNN()

In [11]:
model = CNN()


# optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
# loss_function = tf.keras.losses.sparse_categorical_crossentropy

# y_pred = model(train_x[:100])
# print(y_pred)

# with tf.GradientTape() as tape:
    # y_pred = model(train_x)
    # print(y_pred.shape, train_y.shape)
    # loss = loss_function(y_pred, train_y)
    # print(loss)


# for i in range(10):
    # with tf.GradientTape() as tape:
    #     y_pred = model(train_x)
    #     print(y_pred)
        # loss = tf.reduce_mean(loss_function(y_pred, train_y))
        # print(loss)
    # grads = tape.gradient(loss, model.variables)
    # optimizer.apply_gradients(grads_and_vars=zip(grads, model.variables))



# model.compile(optimizer='adam',
#               loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#               metrics=['accuracy'])

In [12]:
history = model.fit(train_x, train_y, epochs=10)

RuntimeError: You must compile your model before training/testing. Use `model.compile(optimizer, loss)`.

In [ ]:
pred = np.argmax(model.predict(test_x), axis=-1).reshape(-1, 1)
pred

array([[2],
       [0],
       [9],
       ...,
       [3],
       [9],
       [2]])

## 输出数据

In [ ]:
image_id = np.array(range(1, 28000 + 1)).reshape(-1, 1)
result = pd.DataFrame(data=np.concatenate([image_id, pred], axis=1), columns=['ImageId', 'Label'])

In [ ]:
result.to_csv('submission.csv', index=False)